In [2]:
#Import train dataset
import pandas as pd

pd.read_csv('train.csv')

df_edges_all=pd.read_csv('train.csv')


df_edges= df_edges_all.iloc[:int(len(df_edges_all)*0.8)]

df_edges_test = df_edges_all.iloc[int(len(df_edges_all)*0.8):]
df_edges.head()


,id,id1,id2,label
0,1,9202,9202,1
1,2,410411,460254,0
2,3,211858,312074,1
3,6,253901,504325,0
4,7,415071,63239,0


In [3]:
df_edges_test.head()

,id,id1,id2,label
758585,976717,345657,467621,1
758586,976719,825394,20456,1
758587,976720,720678,447520,0
758588,976721,181991,649051,1
758589,976723,663566,821902,0


In [4]:
dfdesc= pd.read_csv('nodes/nodes.tsv', sep='\t', header=0)



In [5]:
print(dfdesc.shape)
dfdesc=dfdesc.set_index('id')
dfdesc.head()


(837834, 2)


,text
id,
1,{{infobox person | name = clayton jacobson | i...
2,a '''cobra probe''' is a device to measure the...
3,the '''harmon foundation''' was established in...
4,'''structured finance''' is a sector of financ...
5,'''al-shohada'a stadium''' is a multi-use stad...


In [6]:
# TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

dfdesc = dfdesc.fillna("")
corpus = dfdesc.values[:,0]
print(corpus[0])
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)



{{infobox person | name = clayton jacobson | image = | imagesize = | caption = | birth_name = | birth_date = | birth_place = | death_date = <!-- death date then birth --> | death_place = | other_names = | occupation = director, producer, writer, actor, editor | years_active = 1989 – present | spouse = | partner = | website = }} '''clayton jacobson''' (born 26 october 1963) is an australian film director, writer, producer, actor, musician and editor. his debut feature film was'' kenny (2006 film)''. ''kenny'' was released in 2006 in australia to critical acclaim, winning a number of awards. jacobson has also acted in a number of films, including 2010's ''animal kingdom (film)''. he is the brother of ''kenny'' actor shane jacobson. he also plays bass in the appalachian folk band the duck downpickers.


In [8]:
X.shape

(837834, 1655398)

In [9]:
from tqdm import tqdm
tqdm.pandas()

sparsevectors={}

for i in tqdm(range(X.shape[0])):
    sparsevectors[dfdesc.index[i]]=X[i]
    

100%|████████████████████████████| 837834/837834 [00:44<00:00, 19008.46it/s]


In [11]:
import pickle 

with open('text_vectors.pkl', 'wb') as f:
    pickle.dump(sparsevectors, f)

In [14]:
#import Stemmer
#english_stemmer = Stemmer.Stemmer('en')
#class StemmedTfidfVectorizer(TfidfVectorizer):
#    def build_analyzer(self):
#        analyzer = super(TfidfVectorizer, self).build_analyzer()
#        return lambda doc: english_stemmer.stemWords(analyzer(doc))
#    
#vectorizer = StemmedTfidfVectorizer(min_df=1, stop_words='english', analyzer='word', ngram_range=(1,1))
#vectorizer.fit_transform(corpus)

In [15]:
# The existing graph
import networkx as nx
G= nx.Graph()
#add nodes
nodes = pd.unique(df_edges_all[['id1', 'id2']].values.ravel())
G.add_nodes_from(nodes)
#add edges
edges = [(row.id1,row.id2) for row in df_edges[df_edges["label"]==1].itertuples()]
G.add_edges_from(edges)

# keeping all the negative edges so we can compare results
no_edges = [(row.id1,row.id2) for row in df_edges[df_edges["label"]==0].itertuples()]

In [16]:
import math
# Adding methods proposed in the https://link.springer.com/content/pdf/10.1038/s41598-019-57304-y.pdf
def shortest_path(G:nx.Graph,source:int,target:int)->float:
    try:
        p=nx.shortest_path(G,source=source,target=target)
        return len(p)-1
    except nx.NetworkXNoPath:
        return float("inf")

def distance(G:nx.Graph,source:int,target:int)->float:
   '''
   This method can be changed to any distance
   '''
   return shortest_path(G,source,target)

from sklearn.metrics.pairwise import cosine_similarity
def distance_description(G,source:int,target:int)->float:
    '''
    This method can be changed to any distance
    '''
    
    
    sp1=sparsevectors[source]
    sp2=sparsevectors[target]
    
    return cosine_similarity(sp1,sp2)[0][0]


def closeness_centrality(G:nx.Graph, source:int, target:int)->float:
    dxy = nx.shortest_path(G,source=source,target=target)
    return G.number_of_nodes()/shortest_path(G,source,target)

def common_neighbors(G:nx.Graph, source:int, target:int)->list:
    s_neighbors = list(G.adj[source].keys())
    t_neighbors = list(G.adj[target].keys())
    common_neighbors = set(s_neighbors).intersection(t_neighbors)
    return common_neighbors

def CCPA(G:nx.Graph, source:int, target:int,a:float = 0.5)->float:
    '''
    Common Neighbor and Centrality based Parameterized Algorithm
    '''
    return a*closeness_centrality(G,source,target)+(1-a)*len(common_neighbors(G,source,target))

def CND(G:nx.Graph, source:int, target:int)->float:
    '''
    Common Neighbor and Distance
    '''
    cn = common_neighbors(G,source,target)
    if len(cn)>0:
        return (len(cn)+1)/2
    else:
        if distance(G,source,target)==0:
            return 0
        return 1/distance(G,source,target)

def PA(G:nx.Graph, source:int, target:int)->float:
    '''
    Preferential Attachment
    '''
    return G.degree[source]*G.degree[target]

def AA(G:nx.Graph, source:int, target:int)->float:
    '''
    Adamic Adar
    '''
    similarity = 0.0
    for neighbor in common_neighbors(G,source,target):
        degree = G.degree[neighbor]
        if degree > 1:
            similarity += 1 / math.log(degree)
    return similarity

def CN(G:nx.Graph, source:int, target:int)->float:
    '''
    Common Neighbor
    '''
    return len(common_neighbors(G,source,target))

def SI(G:nx.Graph, source:int, target:int)->float:
    '''
    Sorensen Index
    '''
    if (G.degree[source]+G.degree[target]) ==0:
        return 0
    return 2*CN(G,source,target)/(G.degree[source]+G.degree[target])

def JI(G:nx.Graph, source:int, target:int)->float:
    '''
    Jaccard Index
    '''
    s_neighbors = list(G.adj[source].keys())
    t_neighbors = list(G.adj[target].keys())
    common_neighbors = set(s_neighbors).intersection(t_neighbors)
    all_neighbors = set(s_neighbors).union(t_neighbors)
    if len(all_neighbors)==0:
        return 0
    return len(common_neighbors)/len(all_neighbors)

def RA(G:nx.Graph, source:int, target:int)->float:
    '''
    Resource Allocation
    '''
    similarity = 0.0
    for neighbor in common_neighbors(G,source,target):
        degree = G.degree[neighbor]
        if degree > 1:
            similarity += 1 / degree
    return similarity

def HPI(G:nx.Graph, source:int, target:int)->float:
    '''
    Hub Promoted Index
    '''
    if min([G.degree[source],G.degree[target]]) ==0 :
        return 0
    return CN(G,source,target)/min([G.degree[source],G.degree[target]])

def create_metric_vector(tup):
    G=tup[0]
    source= tup[1]
    target= tup[2]
    functions = [CND, PA, AA, CN, SI, JI, RA, HPI,distance_description]
    outputs = []
    for func in functions:
        output = func(G, source, target)
        outputs.append(output)
    return outputs




In [17]:
# metrics names
print("[CND, PA, AA, CN, SI, JI, RA, HPI]")
#This for connected nodes
print(create_metric_vector((G,211858,312074)))
#This is for unconnected nodes
print(create_metric_vector((G,410411,460254)))


[CND, PA, AA, CN, SI, JI, RA, HPI]
[1.0, 18, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.1132247541702719]
[0.0, 0, 0.0, 0, 0.0, 0.0, 0.0, 0, 0.021393415850264996]


In [19]:
# TEST SET
# metrics names
print("[CND, PA, AA, CN, SI, JI, RA, HPI]")
#This for connected nodes
print(create_metric_vector((G,345657,467621)))
#This is for unconnected nodes
print(create_metric_vector((G,720678,447520)))

[CND, PA, AA, CN, SI, JI, RA, HPI]
[0.0, 0, 0.0, 0, 0, 0, 0.0, 0, 0.23386042157952988]
[0.0, 0, 0.0, 0, 0, 0, 0.0, 0, 0.015094458857096907]


In [68]:
from xgboost import XGBClassifier

In [20]:


from multiprocessing import Pool


def makedataset(df):
    label=df.label
    data=[]
    allind=len(df.index)
    counter=0
    tups=[]
    for i in tqdm(range(len(df.index))):
        row = df.iloc[i]

        tups.append((G,row.id1,row.id2))
    with Pool(4) as p:
        data=p.map(create_metric_vector, tups)
    
    datadf=pd.DataFrame(data)
    return datadf,label

def makedatasetapply(df):
    label=df.label
    datadf = df.progress_apply(lambda row: create_metric_vector((G,row['id1'],row['id2'])), axis=1,result_type='expand')
    return datadf,label

In [ ]:
from sklearn.model_selection import train_test_split


import time

start = time.time()
Xtest,Ytest=makedataset(df_edges_test)
end = time.time()
print(end - start)


start = time.time()
Xtest,Ytest=makedatasetapply(df_edges_test)
end = time.time()
print(end - start)



#start = time.time()
#X,Y=makedataset(df_edges)
#end = time.time()
#print(end - start)




100%|████████████████████████████████| 189647/189647 [00:08<00:00, 21410.10it/s]


In [108]:

start = time.time()
Xtest,Ytest=makedatasetapply(df_edges_test)
end = time.time()
print(end - start)

start = time.time()
X,Y=makedatasetapply(df_edges)
end = time.time()
print(end - start)

100%|█████████████████████████████████| 189647/189647 [00:19<00:00, 9720.54it/s]


19.524142026901245


100%|█████████████████████████████████| 758585/758585 [01:18<00:00, 9658.79it/s]

78.55381202697754


In [56]:
from sklearn.metrics import f1_score
model = XGBClassifier()

#X_train, X_test, y_trainn, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
model.fit(X, Y)
preds = model.predict(Xtest)
f1=f1_score(Ytest, preds)
print(f1)

0.007842600825296947


In [58]:
print(preds)

{0, 1}


In [ ]:
!

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::bkcharts==0.2=py39h06a4308_0
  - pytorch/linux-64::torchaudio==0.11.0=py39_cu113
  - defaults/noarch::seaborn==0.11.2=pyhd3eb1b0_0
  - defaults/noarch::altair==4.1.0=py_1
  - defaults/noarch::dask==2021.10.0=pyhd3eb1b0_0
  - pytorch/noarch::captum==0.5.0=0
  - pytorch/linux-64::torchvision==0.12.0=py39_cu113
  - defaults/linux-64::anaconda==custom=py39_1
  - defaults/linux-64::_anaconda_depends==2021.11=py39_0
  - defaults/linux-64::statsmodels==0.13.2=py39h7f8727e_0
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | ^C
failed with repodata from current_repodata.json, will retry with next repodata source.

CondaError: KeyboardInterrupt

